In [9]:
import os
import time
from dotenv import load_dotenv
from pinecone import Pinecone
from IPython.display import display, Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from pinecone import ServerlessSpec



In [10]:
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key)
client = OpenAI()

In [11]:
index_name = "ia-sensor"

if not pc.has_index(index_name):
   pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

# Sem prompt

In [12]:
# Upload do PDFs
file = "files/POP E OPR - 00X.00 - Central ITG 200K - Configuração de Rede .docx (1).pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)

# Dividir texto em chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

# Gera embeddings
embeddings = OpenAIEmbeddings()

# Criação do armazenamento vetorial no Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)

# Configuração do modelo
llm = ChatOpenAI(   
    model='gpt-3.5-turbo',  
    temperature=0.0  
)  

#  Configuração de mecanismo de perguntas e respostas
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)

contexto = "\n".join(texts) 

# Consulta para buscar informações
pergunta = "Como faço para congifurar a central para o cliente?"

pergunta_com_contexto = f"Contexto:\n{contexto}\n\nPergunta: {pergunta}"

resposta = llm(pergunta_com_contexto)

display(Markdown(resposta.content)) 



Para configurar a central ITG200-K para o cliente, siga os seguintes passos:

1. Acesse a interface gráfica da central utilizando o login e senha fornecidos.
2. Na lateral esquerda, acesse a opção "About" para confirmar o SN e MAC Address do equipamento.
3. Confirme se a central está correta e acesse as opções "Network" e "Ethernet".
4. Na opção "Ethernet", insira as configurações solicitadas de acordo com o tipo de rede (DHCP ou estática).
5. Para redes permitidas DHCP, habilite a opção DHCP, salve e aplique o reboot.
6. Para redes estáticas, desabilite o DHCP e configure o IP, Netmask e Default Gateway.
7. Salve as informações e não aplique o reboot conforme solicitado pelo console.
8. Na opção DNS, insira as informações no DNS fixo "Edit Fixed IP DNS Configuration".
9. Salve as informações e aplique o reboot no console.
10. Após o reboot, verifique no display da central se o IP configurado aparece e conecte o cabo de rede no ponto liberado.

Certifique-se de seguir corretamente as instruções e salvar as configurações antes de aplicar o reboot. Se tiver dúvidas ou encontrar dificuldades durante o processo, consulte o procedimento operacional padrão ou entre em contato com o suporte técnico responsável.

# Com prompt

In [14]:
# Upload do PDFs
file = "files/POP E OPR - 00X.00 - Central ITG 200K - Configuração de Rede .docx (1).pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)

# Dividir texto em chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

# generate unique id's

i = 0
uuids = []

while i < len(split_texts):

    i += 1

    uuids.append(f"id{i}")

# add to database

vectorstore.add_documents(documents=split_texts, ids=uuids)

# Gera embeddings
embeddings = OpenAIEmbeddings()

# Criação do armazenamento vetorial no Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)

# Configuração do modelo
llm = ChatOpenAI(   
    model='gpt-3.5-turbo',  
    temperature=0.0  
)  

# Configuração de mecanismo de perguntas e respostas
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)

contexto = "\n".join(texts) 

# Prompt aprimorado para respostas detalhadas e passo a passo
prompt = f"""
Você é um assistente altamente especializado em responder dúvidas com base **exclusivamente** nas informações fornecidas no contexto.  

### **Diretrizes para sua resposta:**
- **Extraia e interprete** os dados do contexto de maneira inteligente e detalhada.  
- **Explique de forma clara**, organizada e estruturada, garantindo que qualquer pessoa possa compreender.  
- **Utilize um formato passo a passo**, descrevendo cada etapa com precisão.  
- Se houver conceitos importantes, forneça explicações adicionais e exemplos práticos.  
- Caso a informação necessária **não esteja no contexto**, deixe isso claro e **não invente** respostas ou especule.  

### **Contexto fornecido:**
{contexto}

### **Pergunta:**
{pergunta}

### **Formato esperado da resposta:**
1. **Análise Inicial**  
   - [Explique como a pergunta se relaciona ao contexto]  
2. **Extração e Interpretação dos Dados**  
   - [Destaque as informações relevantes do contexto]  
3. **Passo a Passo da Resolução**  
   - **Passo 1:** [Explicação detalhada]  
   - **Passo 2:** [Explicação detalhada]  
   - ...  
4. **Conclusão e Recomendações**  
   - [Resumo final com insights úteis]  

Caso não encontre a resposta no contexto, responda:  
*"Não encontrei informações suficientes no contexto para responder com precisão."*  

Seja claro, objetivo e didático em suas explicações.  
"""

# Consulta para buscar informações
pergunta = "Como faço para configurar ao central para o cliente em campo?"

pergunta_com_contexto = prompt

resposta = llm(pergunta_com_contexto)

display(Markdown(resposta.content))



**Análise Inicial**  
A pergunta está relacionada à configuração da central para o cliente em campo, conforme descrito no procedimento operacional padrão fornecido.

**Extração e Interpretação dos Dados**  
- O procedimento operacional padrão aborda a configuração da rede na central ITG200-K.
- O procedimento é aplicável aos setores de Implantação, Preventiva e Suporte.
- Os passos para a configuração da rede incluem acesso à interface gráfica da central, login, senha, confirmação do SN e MAC Address, acesso às instruções de rede e Ethernet, inserção das configurações solicitadas (DHCP ou estáticas), configuração do DNS fixo, salvamento das informações e aplicação de reboot, se necessário.

**Passo a Passo da Resolução**  
1. **Acesso à Interface Gráfica da Central:**
   - Realize o login na interface gráfica da central ITG200-K.
   - Acesse a opção "About" para confirmar o SN e MAC Address do equipamento.

2. **Configuração da Rede:**
   - Após confirmar a central correta, acesse as instruções "Network" e "Ethernet".
   - Na opção "Ethernet", insira as configurações solicitadas:
     - Para redes permitidas DHCP: habilite o DHCP, salve e aplique o reboot.
     - Para redes estáticas: desabilite o DHCP e configure o IP, Netmask e Default Gateway.
     - Salve as informações.

3. **Configuração do DNS Fixo:**
   - Na instrução DNS, insira as informações no DNS fixo "Edit Fixed IP DNS Configuration".
   - Salve as informações.
   - Aplique reboot no console, se necessário.

4. **Verificação e Conclusão:**
   - Após o reboot, verifique no display da central se o IP configurado aparece.
   - Conecte o cabo de rede no ponto liberado.

**Conclusão e Recomendações**  
Para configurar a central para o cliente em campo, siga os passos detalhados no procedimento operacional padrão, garantindo a correta configuração da rede na central ITG200-K. Certifique-se de seguir todas as etapas e salvar as informações conforme orientado para garantir o funcionamento adequado da rede.